In [ ]:
import firedrake
from firedrake import Constant
Lx, Ly = Constant(2.0), Constant(1.0)
ny = 32
nx = int(float(Lx / Ly)) * ny
mesh = firedrake.RectangleMesh(nx, ny, float(Lx), float(Ly))
degree = 1
temperature_space = firedrake.FunctionSpace(mesh, 'CG', degree)

Ra = Constant(1e6)

In [ ]:
import numpy as np
from numpy import pi as π
from firedrake import sqrt, exp, min_value, max_value

def clamp(z, zmin, zmax):
    return min_value(Constant(zmax), max_value(Constant(zmin), z))

def switch(z):
    return exp(z) / (exp(z) + exp(-z))

ϵ = Constant(1 / nx)
x = firedrake.SpatialCoordinate(mesh)

q = Lx**(7 / 3) / (1 + Lx**4)**(2 / 3) * (Ra / (2 * np.sqrt(π)))**(2/3)
Q = 2 * firedrake.sqrt(Lx / (π * q))
T_u = 0.5 * switch((1 - x[1]) / 2 * sqrt(q / (x[0] + ϵ)))
T_l = 1 - 0.5 * switch(x[1] / 2 * sqrt(q / (Lx - x[0] + ϵ)))
T_r = 0.5 + Q / (2 * np.sqrt(π)) * sqrt(q / (x[1] + 1)) * exp(-x[0]**2 * q / (4 * x[1] + 4))
T_s = 0.5 - Q / (2 * np.sqrt(π)) * sqrt(q / (2 - x[1])) * exp(-(Lx - x[0])**2 * q / (8 - 4 * x[1]))
expr = T_u + T_l + T_r + T_s - Constant(1.5)

T_0 = firedrake.interpolate(
    clamp(expr, 0, 1),
    temperature_space,
)
T = T_0.copy(deepcopy=True)

In [ ]:
import matplotlib.pyplot as plt
def subplots():
    fig, axes = plt.subplots()
    axes.set_aspect('equal')
    axes.get_xaxis().set_visible(False)
    axes.get_yaxis().set_visible(False)
    axes.set_xlim(0, float(Lx))
    axes.set_ylim(0, float(Ly))
    return fig, axes

fig, axes = subplots()
firedrake.tripcolor(T, cmap='inferno', axes=axes);

In [ ]:
pressure_space = firedrake.FunctionSpace(mesh, 'CG', 1)
velocity_space = firedrake.VectorFunctionSpace(mesh, 'CG', 2)
Z = velocity_space * pressure_space
z = firedrake.Function(Z)
u, p = firedrake.split(z)

In [ ]:
from firedrake import inner, sym, grad, div, dx, as_vector
μ = Constant(1)
ε = sym(grad(u))
τ = 2 * μ * ε

g = as_vector((0, -1))
f = -Ra * T * g
J = (0.5 * inner(τ, ε) - p * div(u) - inner(f, u)) * dx

F = firedrake.derivative(J, z)
bc = firedrake.DirichletBC(Z.sub(0), as_vector((0, 0)), 'on_boundary')

In [ ]:
from firedrake import (
    NonlinearVariationalProblem as Problem,
    NonlinearVariationalSolver as Solver,
    MixedVectorSpaceBasis,
    VectorSpaceBasis,
)
basis = VectorSpaceBasis(constant=True)
nullspace = MixedVectorSpaceBasis(Z, [Z.sub(0), basis])
stokes_problem = Problem(F, z, bc)
parameters = {
    'solver_parameters': {
        'ksp_type': 'preonly',
        'pc_type': 'lu',
        'pc_factor_mat_solver_type': 'mumps',
    }
}
stokes_solver = Solver(stokes_problem, nullspace=nullspace, **parameters)

In [ ]:
stokes_solver.solve()
fig, axes = subplots()
firedrake.streamplot(
    z.sub(0), axes=axes, resolution=1/40, cmap='inferno', seed=1729
);

In [ ]:
ρ, c, k = Constant(1), Constant(1), Constant(1)

T_n = T.copy(deepcopy=True)
J_mass = 0.5 * ρ * c * (T - T_n)**2 * dx
J_cells = 0.5 * k * inner(grad(T), grad(T)) * dx

δx = mesh.cell_sizes.dat.data_ro[:].min()
umax = z.sub(0).dat.data_ro[:].max()
δt = Constant(δx / umax)
J = J_mass + δt * J_cells
F_diffusive = firedrake.derivative(J, T)

ϕ = firedrake.TestFunction(temperature_space)
F_advective = -δt * ρ * c * T * inner(u, grad(ϕ)) * dx

lower_bc = firedrake.DirichletBC(temperature_space, 1, [3])
upper_bc = firedrake.DirichletBC(temperature_space, 0, [4])
bcs = [lower_bc, upper_bc]
F = F_diffusive + F_advective
temperature_problem = Problem(F, T, bcs)
temperature_solver = Solver(temperature_problem, **parameters)

In [ ]:
import tqdm
final_time = 1.
num_steps = int(final_time / float(δt))
Ts = [T.copy(deepcopy=True)]
zs = [z.copy(deepcopy=True)]
output_freq = 1

for step in tqdm.trange(num_steps):
    temperature_solver.solve()
    stokes_solver.solve()
    T_n.assign(T)
    
    if (step + 1) % output_freq == 0:
        Ts.append(T.copy(deepcopy=True))
        zs.append(z.copy(deepcopy=True))

In [ ]:
%%capture
fig, axes = subplots()
colors = firedrake.tripcolor(
    Ts[0], num_sample_points=4, vmin=0.0, vmax=1.0, cmap='inferno', axes=axes
)

In [ ]:
from matplotlib.animation import FuncAnimation
fn_plotter = firedrake.FunctionPlotter(mesh, num_sample_points=4)
def animate(T):
    colors.set_array(fn_plotter(T))

animation = FuncAnimation(fig, animate, frames=Ts, interval=1e3/24)

In [ ]:
from IPython.display import HTML
HTML(animation.to_html5_video())

In [ ]:
z = zs[-1]
u, p = z.split()
fig, axes = subplots()
firedrake.streamplot(
    u, axes=axes, resolution=1/40, cmap='inferno', seed=1729
);